In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (7431 Fisher Island Dr #7431, Miami Beach, FL ...
2     (9401 Collins Ave #607, Surfside, FL 33154, US...
3     (17749 Collins Ave APT 701, Sunny Isles Beach,...
4     (350 Meridian Ave, Miami Beach, FL 33139, USA,...
5     (3737 Collins Ave s 1503, Miami Beach, FL 3314...
6     (13635 Deering Bay Dr PH 294, Coral Gables, FL...
7     (801 South Pointe Dr APT 205, Miami Beach, FL ...
8     (650 NE 32nd St 4806 4807, Miami, FL 33137, US...
9     (3250 NE 188th St APT 608, Aventura, FL 33180,...
10    (4779 Collins Ave, Miami Beach, FL 33140, USA,...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,7400 Oceanside at Fisher Island\r7431 Fisher I...,11/14/2022,120,"$9,250,000.00",5679,"$2,263.83",Dora Puig,Luxe Living Realty,Maria Oneide Gioia,Luxury Living Realty,7400 Oceanside at Fisher Island 7431 Fisher Is...,,7400 Oceanside at Fisher Island 7431 Fisher Is...,"(7431 Fisher Island Dr #7431, Miami Beach, FL ...","(25.7617, -80.13568599999999, 0.0)",25.761700,-80.135686,0.0
2,Azure Condominium\r9401 Collins Ave 607\rSurfside,11/15/2022,211,"$6,500,000.00",4965,"$1,309.16",Silvia Coltrane,The Corcoran Group,Silvia Coltrane,The Corcoran Group,Azure Condominium 9401 Collins Ave 607 Surfside,Azure Condominium,9401 Collins Ave 607 Surfside,"(9401 Collins Ave #607, Surfside, FL 33154, US...","(25.8838004, -80.12177080000001, 0.0)",25.883800,-80.121771,0.0
3,Mansions at Acqualina\r17749 Collins Ave 701\r...,11/18/2022,75,"$6,400,000.00",4609,"$1,388.59",Richard Goihman,Douglas Elliman,Ryan Mendell,Maxwell E Realty Inc,Mansions at Acqualina 17749 Collins Ave 701 Su...,Mansions at Acqualina,17749 Collins Ave 701 Sunny Isles Beach,"(17749 Collins Ave APT 701, Sunny Isles Beach,...","(25.9400518, -80.12056079999999, 0.0)",25.940052,-80.120561,0.0
4,350 Meridian Condo\r350 Meridian Ave PH\rMiami...,11/18/2022,127,"$5,550,000.00",3000,"$1,850.00",Dina Goldentayer,Douglas Elliman,Vicki Miller,Coldwell Banker Realty,350 Meridian Condo 350 Meridian Ave PH Miami B...,,350 Meridian Condo 350 Meridian Ave PH Miami B...,"(350 Meridian Ave, Miami Beach, FL 33139, USA,...","(25.773326, -80.136329, 0.0)",25.773326,-80.136329,0.0
5,Caribbean Condo\r3737 Collins Ave S-1503\rMiam...,11/15/2022,44,"$2,900,000.00",1621,"$1,789.02",Christopher Zoller,BHHS EWM Realty,Stacy Robins,"Stacy Robins Companies, Inc.",Caribbean Condo 3737 Collins Ave S-1503 Miami ...,Caribbean Condo,3737 Collins Ave S-1503 Miami Beach,"(3737 Collins Ave s 1503, Miami Beach, FL 3314...","(25.810647, -80.122714, 0.0)",25.810647,-80.122714,0.0
6,Deering Bay Condo II\r13635 Deering Bay Dr 294...,11/18/2022,127,"$2,450,000.00",3630,$674.93,Edith Neuwahl,Shelton and Stewart REALTORS,Teresita Bernace,Shelton and Stewart REALTORS,Deering Bay Condo II 13635 Deering Bay Dr 294 ...,Deering Bay Condo II,13635 Deering Bay Dr 294 Coral Gables,"(13635 Deering Bay Dr PH 294, Coral Gables, FL...","(25.6362123, -80.29110050000001, 0.0)",25.636212,-80.291101,0.0
7,Marea Condo\r801 S Pointe Dr 205\rMiami Beach,11/15/2022,135,"$2,150,000.00",1728,"$1,244.21",Joonok Yang,NB Elite Realty,Daniel Hechtkopf,Douglas Elliman,Marea Condo 801 S Pointe Dr 205 Miami Beach,Marea Condo,801 S Pointe Dr 205 Miami Beach,"(801 South Pointe Dr APT 205, Miami Beach, FL ...","(25.7686007, -80.1361615, 0.0)",25.768601,-80.136162,0.0
8,Paraiso Bay\r650 NE 32 St 4806-4807\rMiami,11/14/2022,105,"$1,825,000.00",2603,$811.83,Caroline Perez,Keller Williams Miami Beach Realty,Christina Fernandez,"ACC Realty, Inc.",Paraiso Bay 650 NE 32 St 4806-4807 Miami,Paraiso Bay,650 NE 32 St 4806-4807 Miami,"(650 NE 32nd St 4806 4807, Miami, FL 33137, US...","(25.8069485, -80.1862675, 0.0)",25.806949,-80.186267,0.0
9,Echo Aventura\r3250 NE 188 St 608\rAventura,11/16/2022,73,"$1,700,000.00",2630,$646.39,Ruben Alfonso,Beachfront Realty Inc,Tatiana Dzenis,Vera Realty LLC,Echo Aventura 3250 NE 188 St 608 Aventura,Echo Aventura,3250 NE 188 St 608 Aventura,"(3250 NE 188th St APT 608, Aventura, FL 33180,...","(25.9490014, -80.1372188, 0.0)",25.949001,-80.137219,0.0
10,0Blue Diamond Condo\r4779 Collins Ave 2408\rMi...,11/17/2022,47,"$1,500,000.00",1490,"$1,006.71",Julio Oliveira,Elite International Realty Inc,Michelle Dehez,Gary Hennes Realtors,0Blue Diamond Condo 4779 Collins Ave 2408 Miam...,,0Blue Diamond Condo 4779 Collins Ave 2408 Miam...,"(4779 Collins Ave, Miami Beach, FL 33140, USA,...","(25.8233175, -80.1215857, 0.0)",25.823318,-80.121586,0.0

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.at[1,'building_name']=('7400 Oceanside')
df.at[4,'building_name']=('350 Meridian')
df.at[10,'building_name']=('Blue Diamond')

In [11]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only', 'loc',
       'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [13]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('November 13th - November 19th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [14]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [15]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [16]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_111922
